In [1]:
from pyspark.sql import SparkSession
 
# Create a Spark session
spark = SparkSession.builder.appName("DataLoadingExample").getOrCreate()
 
# Get the input data location from the command line or configuration
input_data_location = "data/1987.csv"
 
# Load the data into a PySpark DataFrame
df = spark.read.csv(input_data_location, header=True, inferSchema=True)

# List of columns to be removed
columns_to_remove = ['ArrTime', 'ActualElapsedTime', 'AirTime', 'TaxiIn', 'Diverted',
                     'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
 
# Select columns that are NOT in the 'columns_to_remove' list
df = df.select([col for col in df.columns if col not in columns_to_remove])
 
# Show the resulting DataFrame after removing specified columns
df.show()



24/01/10 13:06:54 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/01/10 13:06:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/10 13:06:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|CancellationCode|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|1987|   10|        14|        3|    741|       730|       849|           PS|     1451|     NA|            79|      23|      11|   SAN| SFO|     447|     NA|        0|              NA|
|1987|   10|        15|        4|    729|       730|       849|           PS|     1451|     NA|            79|      14|      -1|   SAN| SFO|     447|     NA|        0|              NA|
|1987|   10|        17|        6|    741|       730|       849|           P

In [2]:
df = df.na.replace('NA', None)

# Print number of unique values in a column
for col in df.columns:
    if df.select(col).distinct().count() == 1:
        print("Column '{}' has only one unique value".format(col))
        # Remove columns with only one unique value
        df = df.drop(col)

# Identify numerical and categorical columns
categorical_cols = ['UniqueCarrier',  'Dest', 'Origin']
numerical_cols = [x for x in df.columns if x not in categorical_cols]

target_var = 'ArrDelay'

# Remove the target variable 'ArrDelay' from the lists
if target_var in numerical_cols:
    numerical_cols.remove(target_var)
if target_var in categorical_cols:
    categorical_cols.remove(target_var)

# Print the lists
print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F
from pyspark.ml import Pipeline


# Label encode categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_encoded") for col in categorical_cols]
pipeline = Pipeline(stages=indexers)
df_encoded = pipeline.fit(df).transform(df)

# Drop the original categorical columns
df_encoded = df_encoded.drop(*categorical_cols)

# Show the DataFrame with label-encoded categorical columns
df_encoded.show()

Column 'Year' has only one unique value


Column 'TailNum' has only one unique value


Column 'TaxiOut' has only one unique value


Column 'CancellationCode' has only one unique value
Numerical Columns: ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime', 'FlightNum', 'CRSElapsedTime', 'DepDelay', 'Distance', 'Cancelled']
Categorical Columns: ['UniqueCarrier', 'Dest', 'Origin']


+-----+----------+---------+-------+----------+----------+---------+--------------+--------+--------+--------+---------+---------------------+------------+--------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Distance|Cancelled|UniqueCarrier_encoded|Dest_encoded|Origin_encoded|
+-----+----------+---------+-------+----------+----------+---------+--------------+--------+--------+--------+---------+---------------------+------------+--------------+
|   10|        14|        3|    741|       730|       849|     1451|            79|      23|      11|     447|        0|                 11.0|         5.0|          28.0|
|   10|        15|        4|    729|       730|       849|     1451|            79|      14|      -1|     447|        0|                 11.0|         5.0|          28.0|
|   10|        17|        6|    741|       730|       849|     1451|            79|      29|      11|     447|        0|                 11.0|   

+-----+----------+---------+-------+----------+----------+---------+-------+--------------+--------+--------+--------+-------+---------+----------------+---------------------+------------+--------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Distance|TaxiOut|Cancelled|CancellationCode|UniqueCarrier_encoded|Dest_encoded|Origin_encoded|
+-----+----------+---------+-------+----------+----------+---------+-------+--------------+--------+--------+--------+-------+---------+----------------+---------------------+------------+--------------+
|   10|        14|        3|    741|       730|       849|     1451|   NULL|            79|      23|      11|     447|   NULL|        0|            NULL|                 11.0|         5.0|          28.0|
|   10|        15|        4|    729|       730|       849|     1451|   NULL|            79|      14|      -1|     447|   NULL|        0|            NULL|                 11.0|         


Percentage of null values for each column:
Month: 0.00%
DayofMonth: 0.00%
DayOfWeek: 0.00%
DepTime: 0.00%
CRSDepTime: 0.00%
CRSArrTime: 0.00%
UniqueCarrier: 0.00%
FlightNum: 0.00%
TailNum: 0.00%
CRSElapsedTime: 0.00%
ArrDelay: 0.00%
DepDelay: 0.00%
Origin: 0.00%
Dest: 0.00%
Distance: 0.00%
TaxiOut: 0.00%
Cancelled: 0.00%
CancellationCode: 0.00%


In [3]:
# Close context
spark.stop()